# Lab | Web Scraping

Welcome to the "Books to Scrape" Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the data available on Books to Scrape, an online platform showcasing a wide variety of books. As data analyst, you have been tasked with scraping a specific subset of book data from Books to Scrape to assist publishing companies in understanding the landscape of highly-rated books across different genres. Your insights will help shape future book marketing strategies and publishing decisions.

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The publishing industry, much like others, utilizes data analytics to understand market trends, reader preferences, and the performance of books based on factors such as genre, author, and ratings. Books to Scrape serves as a rich source of such data, offering detailed information about a diverse range of books, making it an ideal platform for extracting insights to aid in informed decision-making within the literary world.

**Task**

Your task is to create a Python script using BeautifulSoup and pandas to scrape Books to Scrape book data, focusing on book ratings and genres. The script should be able to filter books with ratings above a certain threshold and in specific genres. Additionally, the script should structure the scraped data in a tabular format using pandas for further analysis.

**Expected Outcome**

A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`. The function should scrape book data from the "Books to Scrape" website and return a `pandas` DataFrame with the following columns:

**Expected Outcome**

- A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`.
- The function should return a DataFrame with the following columns:
  - **UPC**: The Universal Product Code (UPC) of the book.
  - **Title**: The title of the book.
  - **Price (£)**: The price of the book in pounds.
  - **Rating**: The rating of the book (1-5 stars).
  - **Genre**: The genre of the book.
  - **Availability**: Whether the book is in stock or not.
  - **Description**: A brief description or product description of the book (if available).
  
You will execute this script to scrape data for books with a minimum rating of `4.0 and above` and a maximum price of `£20`. 

Remember to experiment with different ratings and prices to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [Books to Scrape](https://books.toscrape.com/)


**Hint**

Your first mission is to familiarize yourself with the **Books to Scrape** website. Navigate to [Books to Scrape](http://books.toscrape.com/) and explore the available books to understand their layout and structure. 

Next, think about how you can set parameters for your data extraction:

- **Minimum Rating**: Focus on books with a rating of 4.0 and above.
- **Maximum Price**: Filter for books priced up to £20.

After reviewing the site, you can construct a plan for scraping relevant data. Pay attention to the details displayed for each book, including the title, price, rating, and availability. This will help you identify the correct HTML elements to target with your scraping script.

Make sure to build your scraping URL and logic based on the patterns you observe in the HTML structure of the book listings!


---

**Best of luck! Immerse yourself in the world of books, and may the data be with you!**

**Important Note**:

In the fast-changing online world, websites often update and change their structures. When you try this lab, the **Books to Scrape** website might differ from what you expect.

If you encounter issues due to these changes, like new rules or obstacles preventing data extraction, don’t worry! Get creative.

You can choose another website that interests you and is suitable for scraping data. Options like Wikipedia, The New York Times, or even library databases are great alternatives. The main goal remains the same: extract useful data and enhance your web scraping skills while exploring a source of information you enjoy. This is your opportunity to practice and adapt to different web environments!

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urljoin

BASE_URL = "https://books.toscrape.com/"
CATALOGUE_URL = urljoin(BASE_URL, "catalogue/")

# Map text-based rating to integers
RATING_MAP = {
    "One": 1,
    "Two": 2,
    "Three": 3,
    "Four": 4,
    "Five": 5
}

def get_soup(url):
    response = requests.get(url)
    return BeautifulSoup(response.text, 'html.parser')

In [11]:
def extract_book_details(book_url):
    soup = get_soup(book_url)
    
    # UPC
    upc = soup.find('th', text='UPC').find_next_sibling('td').text.strip()
    
    # Title
    title = soup.h1.text.strip()
    
    # Price
    #price_text = soup.find('p', class_='price_color').text.strip()
    #price = float(price_text.replace('£', ''))

    # Price
    price_text = soup.find('p', class_='price_color').text.strip()
    try:
        price = float(price_text.replace('Â', '').replace('£', '').strip())
    except ValueError:
        price = None  # Or you can choose to skip this book

    #price_clean = ''.join(c for c in price_text if c.isdigit() or c == '.')

    
    # Availability
    availability = soup.find('p', class_='availability').text.strip()
    
    # Description (if available)
    description_tag = soup.select_one('#product_description')
    if description_tag:
        description = description_tag.find_next_sibling('p').text.strip()
    else:
        description = "No description available"
    
    # Genre (second last breadcrumb item)
    breadcrumb = soup.select('ul.breadcrumb li a')
    genre = breadcrumb[2].text.strip() if len(breadcrumb) > 2 else "Unknown"

    # Rating
    rating_class = soup.find('p', class_='star-rating')['class'][1]
    rating = RATING_MAP.get(rating_class, 0)
    
    return {
        'UPC': upc,
        'Title': title,
        'Price (£)': price,
        'Rating': rating,
        'Genre': genre,
        'Availability': availability,
        'Description': description
    }

In [12]:
def scrape_books(min_rating=4, max_price=20.0):
    page_num = 1
    all_books = []
    
    while True:
        page_url = urljoin(CATALOGUE_URL, f'page-{page_num}.html')
        soup = get_soup(page_url)
        books = soup.select('.product_pod')

        if not books:
            break  # End of pages
        
        for book in books:
            relative_url = book.h3.a['href'].replace('../../../', '')
            book_url = urljoin(CATALOGUE_URL, relative_url)
            details = extract_book_details(book_url)
            
            if details['Rating'] >= min_rating and details['Price (£)'] <= max_price:
                all_books.append(details)
        
        page_num += 1

    return pd.DataFrame(all_books)

In [ ]:
df = scrape_books(min_rating=4, max_price=20)
print(df.head())